In [30]:
import torch
from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [31]:
training_data = datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor())

test_data = datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor())

In [32]:
batch_size = 64
# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [39]:
class FashionNet(nn.Module):
    
    def __init__(self,):
        super(FashionNet, self).__init__()
        self.flatten = nn.Flatten()
        
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(), 
            # nn.Linear(512, 512),
            # nn.ReLU(),
            # nn.Linear(512, 512),
            # nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [40]:
model = FashionNet().to(device)
print(model)

FashionNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [41]:
loss_function = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0)

In [42]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # back-propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [43]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [45]:
epochs = 50
from tqdm import trange

for t in trange(epochs):
    print(f'Epoch {t+1} \n----------------------------------------')
    train(train_dataloader, model, loss_function, optim)
    test(test_dataloader, model, loss_function)
    
print('Done \n')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1 
----------------------------------------
loss: 1.183741  [    0/60000]
loss: 1.128579  [12800/60000]
loss: 1.128815  [25600/60000]
loss: 1.124442  [38400/60000]
loss: 1.148059  [51200/60000]


  2%|▏         | 1/50 [00:05<04:29,  5.50s/it]

Test Error: 
 Accuracy: 66.4%, Avg loss: 1.120983 

Epoch 2 
----------------------------------------
loss: 1.082904  [    0/60000]
loss: 1.031069  [12800/60000]
loss: 1.039182  [25600/60000]
loss: 1.043858  [38400/60000]
loss: 1.071384  [51200/60000]


  4%|▍         | 2/50 [00:11<04:27,  5.57s/it]

...

Test Error: 
 Accuracy: 80.5%, Avg loss: 0.566456 

Epoch 49 
----------------------------------------
loss: 0.433421  [    0/60000]
loss: 0.433886  [12800/60000]
loss: 0.477018  [25600/60000]
loss: 0.541974  [38400/60000]
loss: 0.598304  [51200/60000]


 98%|█████████▊| 49/50 [04:45<00:05,  5.70s/it]

Test Error: 
 Accuracy: 80.6%, Avg loss: 0.564135 

Epoch 50 
----------------------------------------
loss: 0.430422  [    0/60000]
loss: 0.431723  [12800/60000]
loss: 0.474453  [25600/60000]
loss: 0.539845  [38400/60000]
loss: 0.596634  [51200/60000]


100%|██████████| 50/50 [04:50<00:00,  5.81s/it]

Test Error: 
 Accuracy: 80.6%, Avg loss: 0.561885 

Done 

